In [1]:
from pprint import pprint
import warnings

warnings.filterwarnings('ignore')

from nlp import Hear2Learn

In [2]:
h2l.app.reset()

NameError: name 'h2l' is not defined

In [3]:
h2l = Hear2Learn() 

In [4]:
with open('example_transcript.txt') as f:
    text = f.read()
summaries = h2l.summarize_text(text)

In [5]:
pprint(summaries)

[{'points': ['Overlearning is the process of studying something after you can '
             'recall it perfectly.',
             'It helps in reinforcing memory and makes recall easier during '
             'comprehensive finals.',
             'Continuing to study a topic after you can recall it perfectly '
             'aids in better recall during exams.'],
  'topic': 'Overlearning'},
 {'points': ['Organizing material can enhance recall.',
             'Clustering is an example of an organizational strategy where '
             'items are grouped based on their type.',
             'This makes it easier to recall the items as compared to '
             'recalling them in order.',
             'Verbal pneumonic techniques are another example of '
             'organizational strategy.',
             'Acrostics and acronyms are examples of verbal pneumonic '
             'techniques.',
             'Acrostics involve using phrases where the first letter of each '
             'word f

In [7]:
quiz = h2l.generate_quiz(str(summaries))

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  4.77it/s]


In [12]:
print(quiz)

In [14]:
temp[0]

{'question': 'What is the concept of overlearning?',
 'options': ['Studying something before you can recall it perfectly.',
  'Studying something after you can recall it perfectly.',
  'Studying something only once.',
  'Studying something randomly.'],
 'answer': 'Studying something after you can recall it perfectly.'}